Prior to loading the dataset into R, it was pre-processed in Excel. The raw elemental intensities were each divided by the Rh column, which scaled each element, and a new column for each element was generated. Then, all amounts less than or equal to 0 were replaced with 0.0001, to avoid any issues with NaNs in the dataset.

# Load libraries

In [1]:
# try installing vctrs fresh, since it is giving me errors
install.packages("vctrs", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(vctrs)

# Dicer library is used to run ensemble clustering, which generates interim labels for unknown samples
install.packages("diceR", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(diceR)

# try installing Biobase, since I got error messages
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("Biobase")
library(Biobase)

# randomForest is the classification system which identifies unknown samples
install.packages("randomForest", dependencies = TRUE, repos = 'http://cran.us.r-project.org')
library(randomForest)


The downloaded binary packages are in
	/var/folders/pr/t9901z0n4z7dbkstysncwp0m0000gn/T//Rtmp9xSnhB/downloaded_packages

The downloaded binary packages are in
	/var/folders/pr/t9901z0n4z7dbkstysncwp0m0000gn/T//Rtmp9xSnhB/downloaded_packages


Bioconductor version 3.11 (BiocManager 1.30.10), R 4.0.2 (2020-06-22)

Installing package(s) 'Biobase'




The downloaded binary packages are in
	/var/folders/pr/t9901z0n4z7dbkstysncwp0m0000gn/T//Rtmp9xSnhB/downloaded_packages


Old packages: 'backports', 'callr', 'covr', 'devtools', 'DT', 'fs', 'glue',
  'htmlwidgets', 'IRkernel', 'jsonlite', 'knitr', 'MASS', 'mgcv', 'nlme',
  'openssl', 'pkgbuild', 'processx', 'ps', 'RcppArmadillo', 'remotes',
  'stringi', 'survival', 'sys', 'usethis', 'withr', 'xfun'

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, 


The downloaded binary packages are in
	/var/folders/pr/t9901z0n4z7dbkstysncwp0m0000gn/T//Rtmp9xSnhB/downloaded_packages


randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:Biobase’:

    combine


The following object is masked from ‘package:BiocGenerics’:

    combine




In [2]:
# set the seed
set.seed(24924)

# Load train dataset

In [3]:
train <- read.csv("AllSamples.csv", header = TRUE)
head(train)

,X,is_known,Vis,Ag,Al,As,Au,Ca,Cu,Fe,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,71.11.1-89.01,Guess,Gravel Cherts,1368,1,0.0001,54,1515,331,5335,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,71.11.1-89.02,Guess,Gravel Cherts,1269,15,55.0000,25,682,300,2670,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,71.11.1-89.03,Guess,Gravel Cherts,1165,32,4.0000,46,607,523,6119,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,71.11.1-89.04,Guess,Gravel Cherts,1299,40,42.0000,45,338,410,5307,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,71.11.1-89.05,Guess,Gravel Cherts,1724,35,0.0001,51,201,301,1745,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,71.11.1-89.06,Guess,Gravel Cherts,1246,139,159.0000,18,188,292,4858,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


In [4]:
# restrict dataset to only important columns: sample id, is_known, and the elemental intensities
train_df = train[,c(1,2,3,32:58)]
# rename X to sample_id
names(train_df)[1] <- "sample_id"
# preview dataset
head(train_df)

,sample_id,is_known,Vis,Ag.Rh,Al.Rh,As.Rh,Au.Rh,Ca.Rh,Cu.Rh,Fe.Rh,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,71.11.1-89.01,Guess,Gravel Cherts,0.10581683,0.000077400,7.740000e-09,0.004176980,0.11718750,0.02560334,0.4126702,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,71.11.1-89.02,Guess,Gravel Cherts,0.10531120,0.001244813,4.564315e-03,0.002074689,0.05659751,0.02489627,0.2215768,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,71.11.1-89.03,Guess,Gravel Cherts,0.09514864,0.002613525,3.266910e-04,0.003756942,0.04957530,0.04271480,0.4997550,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,71.11.1-89.04,Guess,Gravel Cherts,0.10741751,0.003307699,3.473084e-03,0.003721161,0.02795005,0.03390391,0.4388489,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,71.11.1-89.05,Guess,Gravel Cherts,0.13916694,0.002825315,8.070000e-09,0.004116887,0.01622538,0.02429771,0.1408621,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,71.11.1-89.06,Guess,Gravel Cherts,0.09852139,0.010990749,1.257215e-02,0.001423262,0.01486518,0.02308848,0.3841227,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


# Choose a clustering method

Please see the notebooks "cluster_alg_selection.ipynb" & "cluster_alg_selection_2.ipynb" for more information on this step. The result of those two notebooks is that we selected the gmm algorithm with 5 clusters to create the labels for the artifacts.

# Run ensemble clustering on train dataset

In [5]:
cluster <- consensus_cluster(train_df[,4:30], nk=2:5, p.item=1, reps=1, 
                             algorithms=c("gmm"), scale = FALSE)


Clustering Algorithm 1 of 1: GMM (k = 3) [=========>-----------]  50% eta: 48s

Clustering Algorithm 1 of 1: GMM (k = 4) [===============>-----]  75% eta:  1m

Clustering Algorithm 1 of 1: GMM (k = 5) [=====================] 100% eta:  0s




In [6]:
# save the results of the algorithm as a dataframe
gmm <- cluster[,,"GMM",1:4]
head(gmm)

,2,3,4,5
1,2,1,4,1
2,2,2,2,2
3,2,2,2,2
4,2,2,2,2
5,2,2,2,2
6,2,2,2,2


In [7]:
# Each number in the table refers to the group that each sample has been assigned to.
# Edit each value in the tables by adding "Group_" to each of the numbers so that they are strings and can be counted, not summed, by crosstab
gmm[,1:4] <- paste("Group", gmm[,1:4], sep = "_")
# turn this matrix into a dataframe
gmm <- as.data.frame(gmm)
head(gmm)

,2,3,4,5
,<chr>,<chr>,<chr>,<chr>
1,Group_2,Group_1,Group_4,Group_1
2,Group_2,Group_2,Group_2,Group_2
3,Group_2,Group_2,Group_2,Group_2
4,Group_2,Group_2,Group_2,Group_2
5,Group_2,Group_2,Group_2,Group_2
6,Group_2,Group_2,Group_2,Group_2


In [8]:
# Assign the appropriate labels to each of the Groups
# first add the vis & is_known columns back in
gmm$is_known <- train$is_known
gmm$vis <- train$Vis
# then create an ftable with the sums of each of the known labels
gmm_raw <- gmm[gmm$is_known == "Known",c(4,6)]
gmm_raw <- ftable(gmm_raw[])
gmm_raw <- as.data.frame(gmm_raw)
gmm_raw

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Alibates,5
Group_2,Alibates,16
Group_3,Alibates,0
Group_4,Alibates,2
Group_5,Alibates,0
Group_1,Edwards Plateau Chert,21
Group_2,Edwards Plateau Chert,46
Group_3,Edwards Plateau Chert,0
Group_4,Edwards Plateau Chert,0


In [9]:
gmm_art <- gmm[gmm$is_known != "Known",c(4,6)]
gmm_art <- ftable(gmm_art[])
gmm_art <- as.data.frame(gmm_art)
gmm_art

X5,vis,Freq
<fct>,<fct>,<int>
Group_1,Agate,8
Group_2,Agate,35
Group_3,Agate,0
Group_4,Agate,4
Group_5,Agate,3
Group_1,Alibates,1
Group_2,Alibates,2
Group_3,Alibates,0
Group_4,Alibates,0


In [10]:
# add the results back into the train_df
train_df$gmm_label <- gmm[,4]
head(train_df)

,sample_id,is_known,Vis,Ag.Rh,Al.Rh,As.Rh,Au.Rh,Ca.Rh,Cu.Rh,Fe.Rh,⋯,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh,gmm_label
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,71.11.1-89.01,Guess,Gravel Cherts,0.10581683,0.000077400,7.740000e-09,0.004176980,0.11718750,0.02560334,0.4126702,⋯,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09,Group_1
2,71.11.1-89.02,Guess,Gravel Cherts,0.10531120,0.001244813,4.564315e-03,0.002074689,0.05659751,0.02489627,0.2215768,⋯,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09,Group_2
3,71.11.1-89.03,Guess,Gravel Cherts,0.09514864,0.002613525,3.266910e-04,0.003756942,0.04957530,0.04271480,0.4997550,⋯,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03,Group_2
4,71.11.1-89.04,Guess,Gravel Cherts,0.10741751,0.003307699,3.473084e-03,0.003721161,0.02795005,0.03390391,0.4388489,⋯,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03,Group_2
5,71.11.1-89.05,Guess,Gravel Cherts,0.13916694,0.002825315,8.070000e-09,0.004116887,0.01622538,0.02429771,0.1408621,⋯,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09,Group_2
6,71.11.1-89.06,Guess,Gravel Cherts,0.09852139,0.010990749,1.257215e-02,0.001423262,0.01486518,0.02308848,0.3841227,⋯,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03,Group_2


In [11]:
# Change the labels from Group_n to the appropriate label
# known samples retain their labels, guesses get the new labels from the clustering method
# excepting the El Sauz CHert labels, because those are not actually guesses, we know them to be ESC because of their characteristics

train_df$gmm_label[train_df$is_known == "Known" & train_df$Vis == "El Sauz Chert"] <- "ESC"
train_df$gmm_label[train_df$is_known == "Known" & train_df$Vis == "Oman Chert"] <- "Oman"
train_df$gmm_label[train_df$is_known == "Known" & train_df$Vis == "Edwards Plateau Chert"] <- "EPC"
train_df$gmm_label[train_df$is_known == "Known" & train_df$Vis == "Knife River Flint"] <- "KRF"
train_df$gmm_label[train_df$is_known == "Known" & train_df$Vis == "Alibates"] <- "Alibates"
train_df$gmm_label[train_df$is_known == "Guess" & train_df$Vis == "El Sauz Chert"] <- "ESC"
train_df$gmm_label[train_df$is_known == "Guess" & train_df$gmm_label == "Group_1"] <- "Chert_2"
train_df$gmm_label[train_df$is_known == "Guess" & train_df$gmm_label == "Group_2"] <- "Chert_1"
train_df$gmm_label[train_df$is_known == "Guess" & train_df$gmm_label == "Group_3"] <- "Igneous"
train_df$gmm_label[train_df$is_known == "Guess" & train_df$gmm_label == "Group_4"] <- "ESC"
train_df$gmm_label[train_df$is_known == "Guess" & train_df$gmm_label == "Group_5"] <- "Chert_3"

head(train_df)

,sample_id,is_known,Vis,Ag.Rh,Al.Rh,As.Rh,Au.Rh,Ca.Rh,Cu.Rh,Fe.Rh,⋯,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh,gmm_label
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,71.11.1-89.01,Guess,Gravel Cherts,0.10581683,0.000077400,7.740000e-09,0.004176980,0.11718750,0.02560334,0.4126702,⋯,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09,Chert_2
2,71.11.1-89.02,Guess,Gravel Cherts,0.10531120,0.001244813,4.564315e-03,0.002074689,0.05659751,0.02489627,0.2215768,⋯,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09,Chert_1
3,71.11.1-89.03,Guess,Gravel Cherts,0.09514864,0.002613525,3.266910e-04,0.003756942,0.04957530,0.04271480,0.4997550,⋯,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03,Chert_1
4,71.11.1-89.04,Guess,Gravel Cherts,0.10741751,0.003307699,3.473084e-03,0.003721161,0.02795005,0.03390391,0.4388489,⋯,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03,Chert_1
5,71.11.1-89.05,Guess,Gravel Cherts,0.13916694,0.002825315,8.070000e-09,0.004116887,0.01622538,0.02429771,0.1408621,⋯,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09,Chert_1
6,71.11.1-89.06,Guess,Gravel Cherts,0.09852139,0.010990749,1.257215e-02,0.001423262,0.01486518,0.02308848,0.3841227,⋯,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03,Chert_1


## Random Forests

In this section we will create two random forest classifications. The first will be run using the labels generated by visual classification, which we know are not optimal. The second will be built with the labels generated by the clustering algorithm selected in the previous section.

1. RF built with vis labels
2. RF built with chosen cluster labels

In [12]:
# 1. RF - vis
# the predicted value (Y) is the Vis column 
RF_vis_Y <- train_df$Vis

# the values we will use to predict are the elemental concentrations
RF_x <- train_df[,4:30]

head(RF_x)

,Ag.Rh,Al.Rh,As.Rh,Au.Rh,Ca.Rh,Cu.Rh,Fe.Rh,K.Rh,Mg.Rh,Mn.Rh,⋯,Si.Rh,Sn.Rh,Sr.Rh,Th.Rh,Ti.Rh,U.Rh,V.Rh,Y.Rh,Zn.Rh,Zr.Rh
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.10581683,0.000077400,7.740000e-09,0.004176980,0.11718750,0.02560334,0.4126702,0.010597153,0.009127475,0.003326114,⋯,0.06234530,0.030244431,0.13691213,2.328280e-02,0.015702351,0.027769183,0.005491955,0.003635520,0.090423886,7.740000e-09
2,0.10531120,0.001244813,4.564315e-03,0.002074689,0.05659751,0.02489627,0.2215768,0.005394191,0.007883817,0.010373444,⋯,0.06373444,0.019170124,0.18995851,1.294606e-02,0.013609959,0.016431535,0.006887967,0.000083000,0.071867220,8.300000e-09
3,0.09514864,0.002613525,3.266910e-04,0.003756942,0.04957530,0.04271480,0.4997550,0.015436132,0.003430252,0.005553741,⋯,0.05545573,0.010944136,0.09882391,7.350539e-03,0.018703038,0.008330611,0.005308723,0.004573669,0.039692911,2.531852e-03
4,0.10741751,0.003307699,3.473084e-03,0.003721161,0.02795005,0.03390391,0.4388489,0.013644257,0.012321178,0.004465393,⋯,0.06011742,0.010667328,0.04589432,1.174233e-02,0.021582734,0.024559663,0.001901927,0.011080791,0.032746217,4.796163e-03
5,0.13916694,0.002825315,8.070000e-09,0.004116887,0.01622538,0.02429771,0.1408621,0.000080700,0.002179529,0.007830158,⋯,0.07111721,0.002502422,0.08411366,6.780756e-03,0.010251857,0.010494026,0.005247013,0.012431385,0.009283177,8.070000e-09
6,0.09852139,0.010990749,1.257215e-02,0.001423262,0.01486518,0.02308848,0.3841227,0.001265122,0.007748873,0.009725627,⋯,0.07108405,0.009646556,0.09575393,7.910000e-09,0.008381434,0.006167471,0.001897683,0.000079100,0.014627975,5.851190e-03


In [13]:
RF_output_vis <- randomForest(y = as.factor(RF_vis_Y), x = RF_x, importance = TRUE, 
                              ntree = 10001, proximity = TRUE)
RF_output_vis


Call:
 randomForest(x = RF_x, y = as.factor(RF_vis_Y), ntree = 10001,      importance = TRUE, proximity = TRUE) 
               Type of random forest: classification
                     Number of trees: 10001
No. of variables tried at each split: 5

        OOB estimate of  error rate: 25.29%
Confusion matrix:
                      Agate Alibates Black Cherts Black Metamorphic
Agate                     1        0            0                 0
Alibates                  0        0            0                 0
Black Cherts              0        0            0                 2
Black Metamorphic         0        0            0                 7
Edwards Plateau Chert     0        0            0                 0
El Sauz Chert             0        0            0                 0
Gravel Cherts             2        0            0                 7
Knife River Flint         0        0            0                 0
Limestone                 0        0            0                 0
Oman C

In [14]:
# 2. RF - gmm classification
# the predicted value (Y) is the gmm_label column 
RF_gmm_Y <- train_df$gmm_label

RF_output_gmm <- randomForest(y = as.factor(RF_gmm_Y), x = RF_x, importance = TRUE, 
                              ntree = 10001, proximity = TRUE)
RF_output_gmm


Call:
 randomForest(x = RF_x, y = as.factor(RF_gmm_Y), ntree = 10001,      importance = TRUE, proximity = TRUE) 
               Type of random forest: classification
                     Number of trees: 10001
No. of variables tried at each split: 5

        OOB estimate of  error rate: 15.22%
Confusion matrix:
         Alibates Chert_1 Chert_2 Chert_3 EPC ESC Igneous KRF Oman class.error
Alibates        0      16       4       0   2   1       0   0    0  1.00000000
Chert_1         0     449      10       0   4   0       0   0    0  0.03023758
Chert_2         0      20     147       1   3   2       0   0    0  0.15028902
Chert_3         0       4      17      36   1   9       0   0    0  0.46268657
EPC             0      36      10       0  25   1       0   0    0  0.65277778
ESC             0       6       3       0   0 347       0   0    0  0.02528090
Igneous         0       0       8       7   0   3      17   0    0  0.51428571
KRF             0       4       5       0   1   1     

In [ ]:
# the error rate on both Alibates & Knife River Flint are very high, 
# it seems that neither of those are very distinctive types of chert

# to see what happens when we remove silicified wood and limestone from the dataset 
# and focus on chert and igneous types, see Lithic_classification_2.ipynb

Load test dataset

In [15]:
test <- read.csv("Test.csv", header = TRUE)